# Quickstarter Notebook
Here you can use `basd` with the small example datasets included in the git repository and see how it works!

Importing packages

In [1]:
import os
import pkg_resources

import basd
from dask.distributed import Client, LocalCluster
import numpy as np
import xarray as xr

Setting Paths

In [2]:
input_dir = pkg_resources.resource_filename('basd', 'data') # '../basd/data'
output_dir = pkg_resources.resource_filename('basd', 'data/output') # '../basd/data/output'

Setting output file names

In [3]:
ba_day_output_file = 'pr_sim_fut_ba_day_coarse_2065-2100.nc'
ba_mon_output_file = 'pr_sim_fut_ba_mon_coarse_2065-2100.nc'
basd_day_output_file = 'pr_sim_fut_basd_day_fine_2065-2100.nc'
basd_mon_output_file = 'pr_sim_fut_basd_mon_fine_2065-2100.nc'

#### Reading NetCDF file
Reading in observational precipitation data from 1979-2014 supplied in the data directory.

In [4]:
pr_obs_hist = xr.open_mfdataset(os.path.join(input_dir, 'pr_obs-hist_fine_1979-2014.nc'), chunks={'time': 100})
pr_sim_hist = xr.open_mfdataset(os.path.join(input_dir, 'pr_sim-hist_coarse_1979-2014.nc'), chunks={'time': 100})
pr_sim_fut = xr.open_mfdataset(os.path.join(input_dir, 'pr_sim-fut_coarse_2065-2100.nc'), chunks={'time': 100})

### Running Bias Adjustment
#### Initializing our first `Adjustment` object
Here we are providing the three required data arrays and a variable name 'pr' representing the daily mean precipitation variable. Precipitation is assumed to follow a gamma distribution, and thus requires a lower bound and lower threshold parameter. We also set trend preservation mode to be 'mixed'. Thus, we set the parameter object directly specifying:

* Lower bound = 0
* Lower threshold = 0.0000011574
* Trend preservation = mixed
* Distribution = gamma

Otherwise, we are keeping the default parameter values for the bias adjustment.

In [5]:
params = basd.Parameters(lower_bound=0, lower_threshold=0.0000011574,
                         trend_preservation='mixed',
                         distribution='gamma',
                         if_all_invalid_use=0, n_iterations=20)

# NetCDF output encoding
coarse_encoding = {'missing_value': 1e+20, '_FillValue': 1e+20}

#### Running Bias Adjustment
Here we run the bias adjustment for just one grid cell. We pass in a tuple with the indexes which we wish to adjust, (0,0), which in this case correspond to the coordinates 53.5 N, 12.5 E.

In [6]:
# Optional, set dask temp directory which is recommended on remote cluster to set to /scratch/
# dask.config.set({'temporary_directory': '/scratch/'})
with LocalCluster(processes=True, threads_per_worker=1) as cluster, Client(cluster) as client:
    print(client.dashboard_link)        
    # Initializing Bias Adjustment
    ba = basd.init_bias_adjustment(
        pr_obs_hist, pr_sim_hist, pr_sim_fut,
        'pr', params, 1, 1
    )

    # Doing a single location adjustment and plotting
    one_loc_output = basd.adjust_bias_one_location(ba, {'lat':0, 'lon': 0})
    ecdf_fig = one_loc_output.plot_ecdf(log_x=True)

    # Perform adjustment and save at daily resolution
    basd.adjust_bias(
        init_output = ba, output_dir = output_dir,
        day_file = ba_day_output_file, month_file = ba_mon_output_file,
        clear_temp = True, encoding={ 'pr': coarse_encoding}
    )

http://127.0.0.1:8787/status


In [7]:
ecdf_fig

# Statistical Downscaling
We're now going to start to look at downscaling simulated data to observational data resolution using the statistical downscaling algorithm.

In [8]:
obs_fine = xr.open_mfdataset(os.path.join(input_dir, 'pr_obs-hist_fine_1979-2014.nc'), chunks={'time': 100})
# Using the output from the Bias Adjustment procedure
sim_coarse = xr.open_mfdataset(os.path.join(output_dir, ba_day_output_file), chunks={'time': 100})

In [9]:
# NetCDF output encoding
fine_encoding = {'zlib': True,
                   'shuffle': True,
                   'complevel': 5,
                   'fletcher32': False,
                   'contiguous': False,
                   'chunksizes': (1, 4, 4),
                   'dtype': 'float32',
                   'missing_value': 1e+20,
                   '_FillValue': 1e+20
                   }

In [10]:
with LocalCluster(processes=True, threads_per_worker=1) as cluster, Client(cluster) as client:
    print(client.dashboard_link)
    ds = basd.init_downscaling(
        obs_fine,
        sim_coarse,
        'pr',
        params,
        1,
        1
    )
    basd.downscale(init_output = ds,
                   output_dir = output_dir,
                   day_file = basd_day_output_file,
                   month_file = basd_mon_output_file,
                   clear_temp = True,
                   encoding={'pr': fine_encoding})

http://127.0.0.1:8787/status


Task was destroyed but it is pending!
task: <Task pending name='Task-3713' coro=<HTTP1ServerConnection._server_request_loop() running at /Users/prim232/opt/miniconda3/envs/xesmf_env/lib/python3.9/site-packages/tornado/http1connection.py:825> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x172eb9580>()]> cb=[IOLoop.add_future.<locals>.<lambda>() at /Users/prim232/opt/miniconda3/envs/xesmf_env/lib/python3.9/site-packages/tornado/ioloop.py:687]>
Task was destroyed but it is pending!
task: <Task pending name='Task-3910' coro=<RequestHandler._execute() running at /Users/prim232/opt/miniconda3/envs/xesmf_env/lib/python3.9/site-packages/tornado/web.py:1713> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x1723f6c10>()]> cb=[_HandlerDelegate.execute.<locals>.<lambda>() at /Users/prim232/opt/miniconda3/envs/xesmf_env/lib/python3.9/site-packages/tornado/web.py:2361]>
